<h1>Section 1: This section obtains data from the wikipedia page and stores it appropriately in a dataframe</h1>

In [1]:
import pandas as pd

In [2]:
aslist = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [3]:
type(aslist)
len(aslist)

3

In [4]:
df = aslist[0]
df = df[df['Borough'] != 'Not assigned']
df['Neighborhood'] = df['Neighborhood'].str.replace(' / ', ',')
df.reset_index()
df = df[['Postal code','Borough', 'Neighborhood']]
df.reset_index(drop=True,inplace=True)
df

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern,Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill,Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
df.shape

(103, 3)

<h1>Section 2: In this section, we read latitude/longitude data and add it to our dataframe from Section 1</h1>

In [6]:
df3 = pd.read_csv('http://cocl.us/Geospatial_data')
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df.rename(columns={"Postal code":"PostalCode"}, inplace=True)
df3.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
df3.columns

Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')

In [8]:
out = pd.merge(df.reset_index(), df3.reset_index())

In [9]:
out.head()

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,28,M3H,North York,"Bathurst Manor,Wilson Heights,Downsview North",43.754328,-79.442259
1,41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,88,M8V,Etobicoke,"New Toronto,Mimico South,Humber Bay Shores",43.605647,-79.501321


<h1>Section 3: In this section, we will analyse neighborhoods in Toronto and cluster them</h1>

In [1]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: | ^C
failed

CondaError: KeyboardInterrupt



In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: / 

In [ ]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [1]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_toronto

NameError: name 'folium' is not defined

In [ ]:
CLIENT_ID = 'DXOLZNK5E3HB4GQLOFBTMK1KZKZVADQFLFXVXZHRQOHBXKTK' # your Foursquare ID
CLIENT_SECRET = 'UGSJTKYNZOXNZCYPIK0XP2UTADUJZQVFU43VEJDWBALFRSF4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)